In [1]:
import glob
import json
import os
import csv
import zstandard as zstd
import io

KEYWORDS = [
    "Trump",
    "Clinton",
    "election fraud",
    "alt-right",
    "fake news",
    "populism",
    "media bias"
]

SELECTED_COLUMNS = [
    "id", "author", "created_utc", "title", "selftext", "score",
    "num_comments", "subreddit", "permalink", "author_flair_text"
]

In [2]:
def is_relevant(obj, keywords=KEYWORDS):
    text = ""
    for field in ["title", "selftext", "body"]:
        if field in obj and obj[field]:
            text += " " + str(obj[field])
    text = text.lower()
    return any(keyword.lower() in text for keyword in keywords)

def process_zst_files(input_folder, output_csv_zst):
    zst_files = glob.glob(os.path.join(input_folder, '*.zst'))
    if not zst_files:
        print("No .zst files found in the folder.")
        return

    temp_csv = output_csv_zst + '.tmp'
    
    with open(temp_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=SELECTED_COLUMNS, extrasaction='ignore')
        writer.writeheader()  # Write header once

        for zst_file in zst_files:
            print(f"Processing file: {zst_file}")
            with open(zst_file, 'rb') as f:
                dctx = zstd.ZstdDecompressor(max_window_size=2147483648)
                with dctx.stream_reader(f) as reader:
                    text_stream = io.TextIOWrapper(reader, encoding='utf-8')
                    for line in text_stream:
                        try:
                            obj = json.loads(line)
                        except Exception as e:
                            print(f"Error decoding line in {zst_file}: {e}")
                            continue
                        
                        if not is_relevant(obj):
                            continue

                        filtered_obj = {col: obj.get(col, None) for col in SELECTED_COLUMNS}
                        writer.writerow(filtered_obj)

    with open(temp_csv, 'rb') as input_f, open(output_csv_zst, 'wb') as output_f:
        cctx = zstd.ZstdCompressor()
        cctx.copy_stream(input_f, output_f)

    os.remove(temp_csv)
    print(f"Compressed CSV written to {output_csv_zst}")

In [3]:
input_folder = 'submissions'
output_csv_zst = 'submissions.csv.zst'
process_zst_files(input_folder, output_csv_zst)

Processing file: submissions/RS_2016-10.zst
Processing file: submissions/RS_2016-11.zst
Processing file: submissions/RS_2016-12.zst
Compressed CSV written to submissions.csv.zst


In [14]:
import pandas as pd
import zstandard as zstd
import io

def read_csv_zst(file_path):
    with open(file_path, 'rb') as compressed_file:
        dctx = zstd.ZstdDecompressor()
        with dctx.stream_reader(compressed_file) as reader:
            return pd.read_csv(io.BytesIO(reader.read()))

# Usage
df = read_csv_zst("submissions.csv.zst")

/tmp/ipykernel_1446329/1401575638.py:9: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(io.BytesIO(reader.read()))


In [15]:
df.head()

,id,author,created_utc,title,selftext,score,num_comments,subreddit,permalink,author_flair_text
0,55atq5,optionhome,1475280011,CNN - all lying liberal media. Get off you but...,NaN,0,1.0,Conservative,/r/Conservative/comments/55atq5/cnn_all_lying_...,NaN
1,55atqt,TheSaxonFive,1475280017,Hillary Clinton EXPOSED! Voter Fraud using Nev...,NaN,5,0.0,HillaryForPrison,/r/HillaryForPrison/comments/55atqt/hillary_cl...,NaN
2,55atth,sakedrinker123,1475280048,Donald Trump vs Hillary Clinton debate animal ...,NaN,1,1.0,videos,/r/videos/comments/55atth/donald_trump_vs_hill...,NaN
3,55attz,TheSaxonFive,1475280054,Hillary Clinton EXPOSED! Voter Fraud using Nev...,NaN,29,0.0,conspiracy,/r/conspiracy/comments/55attz/hillary_clinton_...,NaN
4,55atu0,michaelconfoy,1475280054,Trump demands Obama promise not to pardon Clinton,NaN,5,0.0,democrats,/r/democrats/comments/55atu0/trump_demands_oba...,NaN


In [16]:
import pandas as pd

subreddits = [
    "politics",
    "Ask_Politics",
    "The_Donald",
    "AltRight",
    "Conservative",
    "liberal"
]

In [19]:
filtered_df = df[df["subreddit"].isin(subreddits)]

filtered_df.head()

,id,author,created_utc,title,selftext,score,num_comments,subreddit,permalink,author_flair_text
0,55atq5,optionhome,1475280011,CNN - all lying liberal media. Get off you but...,NaN,0,1.0,Conservative,/r/Conservative/comments/55atq5/cnn_all_lying_...,NaN
10,55au1z,chcki,1475280152,NY Times declares Ohio no longer 'Bellwether' ...,NaN,26,1.0,The_Donald,/r/The_Donald/comments/55au1z/ny_times_declare...,FL
12,55au3h,trifecta,1475280171,Trump’s bad week is a ‘nightmare’ for the GOP,NaN,930,245.0,politics,/r/politics/comments/55au3h/trumps_bad_week_is...,NaN
16,55auaz,MRAII,1475280260,Breaking News: DNC is low on funds. That is th...,NaN,31,0.0,The_Donald,/r/The_Donald/comments/55auaz/breaking_news_dn...,TX
18,55aueg,lakersfan014,1475280296,Bernie Sanders Points Out The Real Problem Wit...,NaN,137,12.0,politics,/r/politics/comments/55aueg/bernie_sanders_poi...,NaN


In [20]:
filtered_df = filtered_df[filtered_df["author"] != "[deleted]"]

filtered_df.head()

,id,author,created_utc,title,selftext,score,num_comments,subreddit,permalink,author_flair_text
0,55atq5,optionhome,1475280011,CNN - all lying liberal media. Get off you but...,NaN,0,1.0,Conservative,/r/Conservative/comments/55atq5/cnn_all_lying_...,NaN
10,55au1z,chcki,1475280152,NY Times declares Ohio no longer 'Bellwether' ...,NaN,26,1.0,The_Donald,/r/The_Donald/comments/55au1z/ny_times_declare...,FL
12,55au3h,trifecta,1475280171,Trump’s bad week is a ‘nightmare’ for the GOP,NaN,930,245.0,politics,/r/politics/comments/55au3h/trumps_bad_week_is...,NaN
16,55auaz,MRAII,1475280260,Breaking News: DNC is low on funds. That is th...,NaN,31,0.0,The_Donald,/r/The_Donald/comments/55auaz/breaking_news_dn...,TX
18,55aueg,lakersfan014,1475280296,Bernie Sanders Points Out The Real Problem Wit...,NaN,137,12.0,politics,/r/politics/comments/55aueg/bernie_sanders_poi...,NaN


In [21]:
import pandas as pd
import zstandard as zstd
import os

output_csv_zst = "submissions.csv.zst"

temp_csv = "filtered_subreddits.csv"
filtered_df.to_csv(temp_csv, index=False, encoding="utf-8")

with open(temp_csv, 'rb') as input_f, open(output_csv_zst, 'wb') as output_f:
    cctx = zstd.ZstdCompressor()
    cctx.copy_stream(input_f, output_f)

# Remove the temporary CSV file
os.remove(temp_csv)

print(f"Compressed CSV saved as {output_csv_zst}")

Compressed CSV saved as submissions.csv.zst
